# Monitor Traffic Using Encapsulated Remote Switched Port Analyzer (ERSPAN)
###### <sup>Use case: Paban Sarma (pasarma@cisco.com); Developed by: Deepti S. (deepts@cisco.com) Sarah Samuel (sasamuel@cisco.com). </sup>

Networks often need a secure monitoring solution which can be accessed without a direct connection to the network analyzer. ERSPAN is a Cisco proprietary feature that solves this need. ERSPAN encapsulates the captured traffic in GRE headers and allows it to be transported to a remote port across a Layer 3 network.

In this notebook, we will demonstrate ERSPAN in the following ways:

* Port-level ERSPAN - Complete traffic on the port can be sent to the traffic analyzer. This might not be feasible if the volume of traffic is high.

* ACL-based ERSPAN – the suspicious prefix can be filtered using an ACL and sent to the traffic analyzer for deep inspection.

**NETWORK TOPOLOGY:**

![ERSPAN Use Case](topology.png)

Consider the use case where the incoming traffic on the interface 0/0/0/0 of the router **PE1** needs to be monitored. There is no direct connection between PE1 and a traffic analyzer. The Cisco 8000 Series Routers support only a tunnel interface as the ERSPAN destination. So in this case, the network administrator configures a GRE tunnel from PE1 to the traffic analyzer that is connected to PE2. Then he configures an ERSPAN session to mirror the suspected traffic.

#### To enable ERSPAN on the Cisco 8000 Series Routers, we should configure the following:
* GRE Tunnel interface
* Global ERSPAN configurations
* ERSPAN source attachment configurations


This notebook is divided into the following sections:
* [Topology Bring up](#step1)
* [Start Traffic](#step5)
* [Configure GRE tunnel to Mirror Traffic](#step3)
* [Use case 1: Port-level ERSPAN](#step13)
* [Use case 2: ACL-based ERSPAN](#step18)
* [Clean up Emulator Session](#step7)


The code in the notebooks interacts with the Cisco 8000 emulator with the help of the pyVXR python module in order to bring up and configure a virtual network of routers as per the above topology.

## <a name="step1"></a>Topology Bring Up

> <i>In the cell below, the python module **erspan** contains the code to set up the python enviroment, the simulated 2-router topology, the routing protocol and the linux server according to the network topology diagram, above. The **sim.init** API initializes simulated network with the base configuration. The **sim.up** API brings up the emulator as per the network topology. Refer to the **erspan.py** file in this repository to have a look at the IOS XR base configuration. 

> <i>Click the cell below and then click the play button in this notebook to bring up the network topology. The bring up can be slow and can take 10 minutes or more.

In [1]:
# Installing pre-requisite module
!pip install paramiko
!pip install paramiko-expect

In [2]:
# Bring up of simulated network
from erspan import *
sim = Vxr()
sim.no_image_copy=True

sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-27 10:10 output_dir:vxr.out
INFO:pyvxr.vxr:bd8e02377cc7:/home/vxr/notebooks/Put-Technology-to-Work/ERSPAN
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/beu3recu62 sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:rpe1, rpe2, server
INFO:pyvxr.console:rpe2:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe2.log)
INFO:pyvxr.console:rpe1:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe1.log)
INFO:pyvxr.console:rpe1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:rpe1:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:rpe2:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:rpe1:login successful
INFO:pyvxr.bringup:rpe1:wait for IOS XR RUN state
INFO:pyvxr.bringup:rpe1:run state RPs:1 (expected:1) LCs:0 (expected:0)
INFO:pyvxr.bringup:rpe1:all nodes

Sim status:  {'localhost': 'running'}


> Updating the environment variables.

In [3]:
ports = sim.ports()
serveripaddress = str(ports['server']['HostAgent'])
serverport = str(ports['server']['xr_redir22'])
client, interact = set_server_intf(serveripaddress, serverport, "10.9.9.2", "255.255.255.0")

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!


Last login: Tue Apr 27 10:13:45 2021
[root@localhost ~]# ifconfig eth3 10.9.9.2 netmask 255.255.255.0 up
[root@localhost ~]# 

> <i>At this point, console access to the routers and server is available. Preferrably, you can access the simulated router consoles directly from your laptop through ssh, as this notebook will be accessing the telnet consoles.<i>

In [4]:
print('Consoles can be reached by:')
print('PE1:', get_telnet_cmd(sim, 'rpe1'), '\nPE2:', get_telnet_cmd(sim, 'rpe2'),'\nSERVER:', get_telnet_cmd(sim, 'server'))
print('or better:')
print('PE1:', get_ssh_cmd(sim, 'rpe1'), '\nPE2:', get_ssh_cmd(sim, 'rpe2'), '\nSERVER:', get_ssh_cmd(sim, 'server', True))
print('The password is cisco123')

Consoles can be reached by:
PE1: telnet 172.17.0.2 33197 
PE2: telnet 172.17.0.2 45854 
SERVER: telnet 172.17.0.2 42909
or better:
PE1: ssh cisco@172.17.0.2 -p62772 
PE2: ssh cisco@172.17.0.2 -p62513 
SERVER: ssh root@172.17.0.2 -p60940
The password is cisco123


> <i>When the below cell is played the notebook accesses the telnet consoles of the routers and the server in order to configure the ERSPAN feature and verify it.<i>

In [5]:
loginpe1 = telnetlib.Telnet(str(ports['rpe1']['HostAgent']) , str(ports['rpe1']['serial0']))
loginpe2 = telnetlib.Telnet(str(ports['rpe2']['HostAgent']) , str(ports['rpe2']['serial0']))

## <a name="step5"></a>Start Traffic

> <i>Consider 3 traffic streams flowing across the routers.<i>

![ERSPAN Use Case](topology2.png)

> Start traffic by playing the below cell. Wait for a few minutes until the traffic starts

In [6]:
print("Traffic Generator is starting....")

client1, client2, interact1, interact2 = generate_3traffic_streams(serveripaddress, serverport)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


Last login: Tue Apr 27 10:15:50 2021 from gateway
[root@localhost ~]# ifconfig eth1 10.0.0.1 netmask 255.255.255.0 up; ifconfig et h2 10.1.1.1 netmask 255.255.255.0 up
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./t-rex-64 -i
Starting Scapy server....... Scapy server is started
Trying to bind to vfio-pci ...
Trying to compile and bind to igb_uio ...
ERROR: We don't have precompiled igb_uio.ko module for your kernel version.
Will try compiling automatically...
Success.

/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.89 please wait  ... 
 set driver name net_virtio 
 driver capability  :
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec 

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!


Last login: Tue Apr 27 10:16:00 2021 from gateway
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./trex-console
start -f stl/traffic_3st.py -d 1h -m 100pps -p 0

Using 'python3' as Python interpeter


Connecting to RPC server on localhost:4501                   [SUCCESS]


Connecting to publisher server on localhost:4500             [SUCCESS]


Acquiring ports [0, 1]:                                      [SUCCESS]


Server Info:

Server version:   v2.89 @ STL
Server mode:      Stateless
Server CPU:       1 x VXR
Ports count:      2 x 100.0Gbps @ Virtio network device	

-=TRex Console v3.0=-

Type 'help' or '?' for supported actions

trex>start -f stl/traffic_3st.py -d 1h -m 100pps -p 0


> <i>Now that we have started the 3 traffic streams, ensure that all 3 traffic streams are received on the destination port (CE2 in the toplogy diagram). Analyze the streams at the traffic destination by playing the below cell. Look for source address (src) 10.2.2.1, 10.3.3.1 and 10.4.4.1 in the below cell output to identify the 3 streams of traffic. <i>

In [7]:
TREX_PROMPT = '.*trex.*'

interact2.send('service')
interact2.expect(TREX_PROMPT)

# Capturing the traffic received at the destination port
interact2.send('capture monitor start --rx 1')
interact2.expect('.*#3.*', timeout=1)

interact2.send('capture monitor stop')
interact2.expect(TREX_PROMPT)



Removing all streams from port(s) [0._]:                     [SUCCESS]


Attaching 3 streams to port(s) [0._]:                        [SUCCESS]


Starting traffic on port(s) [0._]:                           [SUCCESS]

30.79 [ms]

trex>service

Enabling service mode on port(s): [0, 1]                     capture monitor start --rx 1
[SUCCESS]

7.99 [ms]

trex(service)>capture monitor start --rx 1

Starting stdout capture monitor - verbose: 'low'             [SUCCESS]


*** use 'capture monitor stop' to abort capturing... ***

trex(service)>



#1 Port: 1 ◀── RX


trex(service)>

    Type: UDP, Size: 60 B, TS: 0.01 [sec]


trex(service)>

    Ether(dst='02:b4:d8:7d:f1:05',src='78:7f:d1:56:40:00',type=2048)/IP(version=4,ihl=5,tos=0,len=46,id=1,flags=0,frag=0,ttl=62,proto=17,chksum=25528,src='10.4.4.1',dst='10.1.1.1')/UDP(sport=1025,dport=12,len=26,chksum=42602)/Raw(load=b'xxxxxxxxxxxxxxxxxx')

trex(service)>



#2 Port: 1 ◀── RX


trex(service)>

    Type: UDP, Size: 60 B, TS: 0.03 [sec]

0

## <a name="step3"></a>Configure GRE tunnel to Mirror Traffic

> <i>Since ERSPAN on Cisco 8000 Series Routers supports only tunnel interfaces as the destination, a GRE tunnel should be configured between **PE1** and the analyzer connected to **PE2**. The ip address of the traffic anaylzer must be reachable from **PE1**. In this topology, the traffic analyzer is in the 10.9.9.0/24 network.

![ERSPAN Use Case](topology3.png)

> <i>The below section configures a GRE tunnel on the **PE1** router with the tunnel source as the loopback interface of **PE1** and the tunnel destination as the ip address of the traffic analyzer.<i>

In [8]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")

loginpe1.write(('''
configure
interface tunnel-ip1
ipv4 address 209.165.201.1 255.255.255.252 
tunnel mode gre ipv4
tunnel source 209.165.200.225
tunnel destination 10.9.9.2
commit
root
exit
!SHOW COMMAND TO VERIFY CONFIGURATION
show interface tunnel-ip1    
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',5)
print(line.decode())

Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.

RP/0/RP0/CPU0:PE1#configure
Tue Apr 27 10:18:37.127 UTC
RP/0/RP0/CPU0:PE1(config)#interface tunnel-ip1
RP/0/RP0/CPU0:PE1(config-if)#ipv4 address 209.165.201.1 255.255.255.252 
RP/0/RP0/CPU0:PE1(config-if)#tunnel mode gre ipv4
RP/0/RP0/CPU0:PE1(config-if)#tunnel source 209.165.200.225
RP/0/RP0/CPU0:PE1(config-if)#tunnel destination 10.9.9.2
RP/0/RP0/CPU0:PE1(config-if)#commit
Tue Apr 27 10:18:37.572 UTC
RP/0/RP0/CPU0:PE1(config-if)#root
RP/0/RP0/CPU0:PE1(config)#exit
RP/0/RP0/CPU0:PE1#!SHOW COMMAND TO VERIFY CONFIGURATION
RP/0/RP0/CPU0:PE1#show interface tunnel-ip1    
Tue Apr 27 10:18:38.546 UTC
tunnel-ip1 is up, line protocol is up 
  Interface state transitions: 1
  Hardware is Tunnel
  Internet address is 209.165.201.1/30
  MTU 1500 bytes, BW 100 Kbit (Max: 100 Kbit)
     reliability 255/255, txload 0/255, rxload 0/255
  Encapsulation TUNNEL_IP,  loopback not set,
  Last link flapped 00:00:00


## <a name="step13"></a>Use case 1: Port-level ERSPAN
The incoming traffic on the 0/0/0/0 interface of the provider edge (PE-1) router is mirrored to a remote traffic analyzer, which is connected to the the PE-2 router.

![ERSPAN Use Case](topology4.png)


### <a name="step4"></a>Configure Port-level ERSPAN

> <i>The below section configures on the **PE1** router:

> <i>    - the global ERSPAN configurations 

> <i>    - the ERSPAN source attachment configurations

In [9]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")

loginpe1.write(('''
configure
!GLOBAL ERSPAN CONFIGURATION
monitor-session ERSPAN1 ethernet
destination interface tunnel-ip1
root
!ERSPAN SOURCE ATTACHMENT CONFIGURATION
interface FourHundredGigE0/0/0/0
monitor-session ERSPAN1 ethernet direction rx-only port-level
commit
root
exit
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',8)
print(line.decode())


RP/0/RP0/CPU0:PE1#configure
Tue Apr 27 10:18:45.265 UTC
RP/0/RP0/CPU0:PE1(config)#!GLOBAL ERSPAN CONFIGURATION
RP/0/RP0/CPU0:PE1(config)#monitor-session ERSPAN1 ethernet
RP/0/RP0/CPU0:PE1(config-mon)#destination interface tunnel-ip1
RP/0/RP0/CPU0:PE1(config-mon)#root
RP/0/RP0/CPU0:PE1(config)#!ERSPAN SOURCE ATTACHMENT CONFIGURATION
RP/0/RP0/CPU0:PE1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE1(config-if)#monitor-session ERSPAN1 ethernet direction rx-only port-level
RP/0/RP0/CPU0:PE1(config-if-mon)#commit
Tue Apr 27 10:18:45.684 UTC
RP/0/RP0/CPU0:PE1(config-if-mon)#root
RP/0/RP0/CPU0:PE1(config)#exit
RP/0/RP0/CPU0:PE1#


> <i>Verify the ERSPAN status

In [10]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")

loginpe1.write(('''
show monitor-session ERSPAN1 status internal   
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',4)
print(line.decode())

Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.


RP/0/RP0/CPU0:PE1#show monitor-session ERSPAN1 status internal   
Tue Apr 27 10:19:00.770 UTC
Information from SPAN Manager and MA on all nodes:
Monitor-session ERSPAN1 (ID 0x00000001) (Ethernet)
SPAN Mgr: Destination interface tunnel-ip1 (0x0f00001c)
          Last error: Success
          Tunnel data:
            Mode: GREoIPv4
            Source IP: 209.165.200.225
            Dest IP: 10.9.9.2
            VRF: 
            ToS: 0 (copied)
            TTL: 255
            DFbit: Not set
0/RP0/CPU0: Destination interface tunnel-ip1 (0x0f00001c)
          Tunnel data:
            Mode: GREoIPv4
            Source IP: 209.165.200.225
            Dest IP: 10.9.9.2
            VRF: 
            ToS: 0 (copied)
            TTL: 255
            DFbit: Not set

Information from SPAN EA on all nodes:
Monitor-session 0x00000001 (Ethernet)
0/RP0/CPU0: Name 'ERSPAN1', destination interface tunnel-ip1 (0x

### <a name="step6"></a>Monitor Redirected Traffic on Traffic Analyzer 

> <i>Now that port level ERSPAN is configured, all the packets incoming on **PE1** from the **CE1** are mirrored to the traffic analyzer, the network administrator or the network surveillance tool uses the following **show** commands:<i>

| tshark command              | Description                                             
| --------------              | -----------                          
| tshark -i eth3 -c 3         | Shows a summary of suspicous traffic packets.          
| tshark -i eth3 -c 1 -P -V   | Shows a detailed dump of the captured packet using Wireshark traffic Analyzer.   

> <i>Play the below cell to view the traffic packets that are mirrored to the traffic analyzer. Look for source address 10.2.2.1, 10.3.3.1 and 10.4.4.1 in the below cell output to identify the 3 streams of traffic.<i>


In [11]:
PROMPT = '.*root.*'
interact.send('tshark -i eth3 -c 3')
interact.expect(PROMPT, timeout=10)

tshark -i eth3 -c 3
Running as user "root" and group "root". This could be dangerous.


0

In [13]:
interact.send('tshark -i eth3 -c 1 -P -V')
interact.expect(PROMPT, timeout=15)

tshark -i eth3 -c 1 -P -V
Running as user "root" and group "root". This could be dangerous.
Capturing on 'eth3'
  1 0.000000000     10.2.2.1 -> 10.1.1.1     UDP 110 Source port: blackjack  Destination port: 12
Frame 1: 110 bytes on wire (880 bits), 110 bytes captured (880 bits) on interface 0
    Interface id: 0
    Encapsulation type: Ethernet (1)
    Arrival Time: Apr 27, 2021 10:19:14.653986890 UTC
    [Time shift for this packet: 0.000000000 seconds]
    Epoch Time: 1619518754.653986890 seconds
    [Time delta from previous captured frame: 0.000000000 seconds]
    [Time delta from previous displayed frame: 0.000000000 seconds]
    [Time since reference or first frame: 0.000000000 seconds]
    Frame Number: 1
    Frame Length: 110 bytes (880 bits)
    Capture Length: 110 bytes (880 bits)
    [Frame is marked: False]
    [Frame is ignored: False]
    [Protocols in frame: eth:ip:gre:erspan:eth:ip:udp:data]
Ethernet II, Src: 78:7f:d1:56:40:18 (78:7f:d1:56:40:18), Dst: 02:69:48:2a:76:c9

0

#### Unconfiguring Port-level ERSPAN for the next use case.

In [14]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")

loginpe1.write(('''
configure
interface FourHundredGigE0/0/0/0
no monitor-session ERSPAN1 ethernet direction rx-only port-level
commit
root
exit
show monitor-session ERSPAN1 status internal
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',8)
print(line.decode())


RP/0/RP0/CPU0:PE1#configure
t
exit
show monitor-session ERSPAN1 status internal
Tue Apr 27 10:19:54.580 UTC
RP/0/RP0/CPU0:PE1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE1(config-if)#no monitor-session ERSPAN1 ethernet direction rx-only port-level
RP/0/RP0/CPU0:PE1(config-if)#commit
Tue Apr 27 10:19:54.985 UTC
RP/0/RP0/CPU0:PE1(config-if)#root
RP/0/RP0/CPU0:PE1(config)#exit
RP/0/RP0/CPU0:PE1#show monitor-session ERSPAN1 status internal
Tue Apr 27 10:19:55.273 UTC
Information from SPAN Manager and MA on all nodes:
Monitor-session ERSPAN1 (ID 0x00000001) (Ethernet)
SPAN Mgr: Destination interface tunnel-ip1 (0x0f00001c)
          Last error: Success
          Tunnel data:
            Mode: GREoIPv4
            Source IP: 209.165.200.225
            Dest IP: 10.9.9.2
            VRF: 
            ToS: 0 (copied)
            TTL: 255
            DFbit: Not set
0/RP0/CPU0: Destination interface tunnel-ip1 (0x0f00001c)
          Tunnel data:
            Mode: GREoIPv4
         

## <a name="step18"></a>Use case 2: ACL-based ERSPAN

Port-level ERSPAN is not recommended when there is high traffic inflow. Using ACL-based ERSPAN, we can mirror specific streams of traffic to the Analyzer for a deeper inspection.

ACLs control whether routed packets are forwarded or blocked at the router interface. The router examines each packet in order to determine whether to forward or drop the packet based on the criteria within the ACL. ACL criteria include:

* Source address of the traffic
* Destination address of the traffic
* Upper-layer protocol

There are three streams of incoming traffic on port 0/0/0/0 of the provider edge (PE1) router. In this use case, one of these streams is mirrored to the traffic analyzer using Access Control List (ACL) based ERSPAN.

![ERSPAN Use Case](topology5.png)


### <a name="step8"></a>Configure ACL-based ERSPAN


> <i>The section below: 
    
> <i> - Configures an ACL which specifies the host IP addresses which should be permitted/denied and captured. 
    
> <i> - Applies the ACL to the incoming interface of PE1 is 0/0/0/0, along with ERSPAN. 
    
> <i>In this case, only the UDP traffic stream from the 10.3.3.0/24 network will be sent to the analyzer.<i>


In [15]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")
loginpe1.write(('''
configure
!ACL CONFIGURATION
ipv4 access-list INGRESS_ACL
10 permit udp 10.2.2.0/24 any
20 permit udp 10.3.3.0/24 any capture
30 permit udp 10.4.4.0/24 any 
commit
root
!APPLY ACL AND ERSPAN TO THE SOURCE INTERFACE
interface FourHundredGigE0/0/0/0
monitor-session ERSPAN1 ethernet direction rx-only port-level acl
ipv4 access-group INGRESS_ACL ingress
commit
root
exit
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',8)
print(line.decode())

Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.

RP/0/RP0/CPU0:PE1#configure
redGigE0/0/0/0
monitor-session ERSPAN1 ethernet direction rx-only port-level acl
ipv4 access-group INGRESS_ACL ingress
commit
root
exit
Tue Apr 27 10:20:43.071 UTC
RP/0/RP0/CPU0:PE1(config)#!ACL CONFIGURATION
RP/0/RP0/CPU0:PE1(config)#ipv4 access-list INGRESS_ACL
RP/0/RP0/CPU0:PE1(config-ipv4-acl)#10 permit udp 10.2.2.0/24 any
RP/0/RP0/CPU0:PE1(config-ipv4-acl)#20 permit udp 10.3.3.0/24 any capture
RP/0/RP0/CPU0:PE1(config-ipv4-acl)#30 permit udp 10.4.4.0/24 any 
RP/0/RP0/CPU0:PE1(config-ipv4-acl)#commit
Tue Apr 27 10:20:43.517 UTC
RP/0/RP0/CPU0:PE1(config-ipv4-acl)#root
RP/0/RP0/CPU0:PE1(config)#!APPLY ACL AND ERSPAN TO THE SOURCE INTERFACE
RP/0/RP0/CPU0:PE1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE1(config-if)#monitor-session ERSPAN1 ethernet direction rx-only port-level acl
RP/0/RP0/CPU0:PE1(config-if)#ipv4 access-group INGRESS_ACL ingress
RP/0/RP0/C

> <i> Verify the status of ERSPAN

In [16]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")
loginpe1.write(('''
show monitor-session ERSPAN1 status internal  
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',4)
print(line.decode())


RP/0/RP0/CPU0:PE1#show monitor-session ERSPAN1 status internal  
Tue Apr 27 10:20:52.687 UTC
Information from SPAN Manager and MA on all nodes:
Monitor-session ERSPAN1 (ID 0x00000001) (Ethernet)
SPAN Mgr: Destination interface tunnel-ip1 (0x0f00001c)
          Last error: Success
          Tunnel data:
            Mode: GREoIPv4
            Source IP: 209.165.200.225
            Dest IP: 10.9.9.2
            VRF: 
            ToS: 0 (copied)
            TTL: 255
            DFbit: Not set
0/RP0/CPU0: Destination interface tunnel-ip1 (0x0f00001c)
          Tunnel data:
            Mode: GREoIPv4
            Source IP: 209.165.200.225
            Dest IP: 10.9.9.2
            VRF: 
            ToS: 0 (copied)
            TTL: 255
            DFbit: Not set

Information from SPAN EA on all nodes:
Monitor-session 0x00000001 (Ethernet)
0/RP0/CPU0: Name 'ERSPAN1', destination interface tunnel-ip1 (0x0f00001c)
Platform, 0/RP0/CPU0:

  Monitor Session ID: 1
  TransportNH ID: 1
  GRE Destinati

### <a name="step6"></a>Monitor Redirected Traffic Stream on Traffic Analyzer 

> <i>Now that the packets incoming on **PE1** from the **PE2** router are mirrored to the traffic analyzer, the network administrator or the network surveillance tool uses the following **show** commands:<i>
    
| tshark command              | Description                                             
| --------------              | -----------                          
| tshark -i eth3 -c 3         | Shows a summary of suspicous traffic packets.          
| tshark -i eth3 -c 1 -P -V   | Shows a detailed dump of the captured packet using Wireshark traffic Analyzer.   

> <i>Play the below cell to view the traffic packets that are mirrored to the traffic analyzer. Only the traffic stream with source address 10.3.3.1 should be present at the analyzer.<i>

In [17]:
PROMPT = '.*root@localhost.*'
interact.send('tshark -i eth3 -c 3')
interact.expect(PROMPT, timeout=10)

Running as user "root" and group "root". This could be dangerous.
Capturing on 'eth3'
  1 0.000000000     10.4.4.1 -> 10.1.1.1     UDP 110 Source port: blackjack  Destination port: 12
Frame 1: 110 bytes on wire (880 bits), 110 bytes captured (880 bits) on interface 0
    Interface id: 0
    Encapsulation type: Ethernet (1)
    Arrival Time: Apr 27, 2021 10:19:28.354042530 UTC
    [Time shift for this packet: 0.000000000 seconds]
    Epoch Time: 1619518768.354042530 seconds
    [Time delta from previous captured frame: 0.000000000 seconds]
    [Time delta from previous displayed frame: 0.000000000 seconds]
    [Time since reference or first frame: 0.000000000 seconds]
    Frame Number: 1
    Frame Length: 110 bytes (880 bits)
    Capture Length: 110 bytes (880 bits)
    [Frame is marked: False]
    [Frame is ignored: False]
    [Protocols in frame: eth:ip:gre:erspan:eth:ip:udp:data]
Ethernet II, Src: 78:7f:d1:56:40:18 (78:7f:d1:56:40:18), Dst: 02:69:48:2a:76:c9 (02:69:48:2a:76:c9)
    D

0

In [18]:
interact.send('tshark -i eth3 -c 1 -P -V')
interact.expect(PROMPT, timeout=15)

Running as user "root" and group "root". This could be dangerous.
Capturing on 'eth3'
  1 0.000000000     10.3.3.1 -> 10.1.1.1     UDP 110 Source port: blackjack  Destination port: 12
  2 0.030985273     10.3.3.1 -> 10.1.1.1     UDP 110 Source port: blackjack  Destination port: 12
  3 0.060113403     10.3.3.1 -> 10.1.1.1     UDP 110 Source port: blackjack  Destination port: 12
3 packets captured
[root@localhost ~]# 

0

## <a name="step7"></a>Clean up Emulator Session

> <i>Bring down the server sessions and the emulator once done.

In [19]:
client.close()
client1.close()
client2.close()
loginpe1.close()
loginpe2.close()

In [20]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/beu3recu62)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [21]:
sim.clean()
# Clean up sim scratch space
shutil.rmtree(sim_dir)

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/beu3recu62)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


If you have any comments or suggestions about this notebook or ideas for anymore notebooks, please send an email to sasamuel@cisco.com